In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from docx import Document
import os
from PIL import Image, ImageOps
import io
import numpy as np
import fitz  # PyMuPDF
from skimage import feature, color, exposure
import pandas as pd

from time import sleep
from tqdm import tqdm

# PDF图片提取及中心坐标提取
## 检测图片是否为反色

**函数名称：** is_image_inverted_combined

**目标：** 综合使用三种不同的方法来检测图像是否被反色。

### 亮度分布检测
**原理：** 该方法计算图像中明亮像素（值大于128）的比例，并与预设的亮度阈值进行比较。

如果明亮像素的比例超过阈值，该方法将投票为“反色”。
### 颜色直方图比较
**原理：** 这种方法是通过比较图像的颜色直方图和其反色版本的直方图来工作的。

使用相关系数来测量原始图像直方图和其反色直方图之间的相似性。
如果相关系数超过预设的直方图阈值，该方法将投票为“反色”。
### 边缘检测比较
**原理：** 首先，使用边缘检测算法（如Canny）来提取图像的边缘。然后，比较原始图像和其反色版本的边缘强度。

使用相关系数来测量原始图像的边缘和其反色版本之间的边缘强度相似性。
如果相关系数超过预设的边缘检测阈值，该方法将投票为“反色”。
### 投票机制
每种方法都会基于其独特的检测策略进行投票。
如果至少有两种方法投票为“反色”，那么函数将返回True，表示图像被认为是反色的。否则，返回False。

In [3]:
def is_image_inverted_combined(img, brightness_threshold=0.75, 
                histogram_threshold=0.8, edge_threshold=0.8):
    # Convert image to RGB mode if it's in "P" mode (palette mode)
    
    img = img.convert("RGB")
    
    # Convert image to grayscale
    img_gray = img.convert("L")
    img_gray_np = np.array(img_gray)
    
    # 1. Brightness distribution
    light_fraction = np.mean(img_gray_np > 128)
    brightness_decision = light_fraction > brightness_threshold
#     print(light_fraction)
#     print(brightness_decision)
    
    # 2. Color histogram comparison
    img_hist = exposure.histogram(np.array(img.convert("RGB")), nbins=256)
    inverted_img_hist = exposure.histogram(np.array(ImageOps.invert(img).convert("RGB")), nbins=256)
    hist_correlation = np.corrcoef(img_hist[0], inverted_img_hist[0])[0, 1]
    histogram_decision = hist_correlation > histogram_threshold
#     print(hist_correlation)
#     print(histogram_decision)
    
    # 3. Edge detection comparison
    edges_original = feature.canny(color.rgb2gray(np.array(img)))
    edges_inverted = feature.canny(color.rgb2gray(np.array(ImageOps.invert(img))))
    edge_correlation = np.corrcoef(edges_original.flatten(), edges_inverted.flatten())[0, 1]
    edge_decision = edge_correlation > edge_threshold
#     print(edge_correlation)
#     print(edge_decision)
    
    # Majority voting
    return sum([brightness_decision, histogram_decision, edge_decision]) >= 256

## 检测低信息含量图片

In [4]:
# def is_low_variance_image_in_memory(img, threshold=3000):
#     try:
#         data = np.array(img.convert('RGB'))
#         variance = np.var(data)
#         return variance < threshold
#     except Exception as e:
#         print(f"Error processing image: {e}")
#         return None

## PyMuPDF获取图片及坐标

In [5]:
# 建立dataframe存储坐标信息
coordinate_df = pd.DataFrame(columns=["file_name","x0","y0","x1","y1","centre_coordinate"])

In [6]:
def PyMuPDF_get_img(file_path, subfolder_name = 'PyMu_img_folder', coordinate_df=coordinate_df):
    i = 0
    # 打开PDF并获取名称
    pdf_file = fitz.open(file_path)
    pdf_file_name = os.path.basename(file_path)

    # 检查子目录是否存在，如果不存在则创建
    if not os.path.exists(subfolder_name):
        os.makedirs(subfolder_name)
    
    for page_number in range(len(pdf_file)):
        # 获取页数
        page = pdf_file[page_number]
        
        # 获取图像信息和图像坐标
        image_list = page.get_images(full=True)

        for img_index, img_info in enumerate(image_list):
            
            #获取图片名称
            image_filename = f"{pdf_file_name}_page_{page_number + 1}_img_{img_index + 1}.png"
            
            img_xref = img_info[0]
            base_image = pdf_file.extract_image(img_info[0])
            if isinstance(base_image, dict):
                image_bytes = base_image["image"]
                try:
                    image = Image.open(io.BytesIO(image_bytes))

                    # 获取图像坐标
                    try:
                        image_rect = page.get_image_rects(img_xref)
                        if not image_rect:
                            print(f"Warning: No image rectangle found for xref {img_xref} on page {page_number + 1} in PDF {pdf_file_name}. Skipping this image.")
                            continue
                        rect = image_rect[0]
                        x0, y0, x1, y1 = rect.x0, rect.y0, rect.x1, rect.y1
                        center_coord = (x0 + x1)/2 , (y0 + y1)/2
                    except Exception as e:
                        print(f"Error getting image rectangle for xref {img_xref} on page {page_number + 1} in PDF {pdf_file_name}: {str(e)}. \nSkipping this image.")

                    # 判断图片大小，过滤过小的图片
                    if abs(x0-x1) < 100:
                        continue

                    # 打开图像并检查颜色模式
                    image = Image.open(io.BytesIO(image_bytes))
                    if image.mode == "CMYK":
                        image = image.convert("RGB")

                    # 检查图片是否反色
                    if is_image_inverted_combined(image):
                        print('Inverted image detected: {} is inverted.'.format(image_filename))
                        image = image.convert("RGB")  # Ensure the image is in RGB mode
                        image = ImageOps.invert(image)

                    # 图片保存
                    image_filepath = os.path.join(subfolder_name, image_filename)
                    image.save(open(image_filepath, "wb"), "PNG")

                    # 坐标保存
                    info = pd.Series([image_filename, x0, y0, x1, y1, center_coord],
                                    index=["file_name","x0","y0","x1","y1","centre_coordinate"])
                    coordinate_df = pd.concat([coordinate_df, pd.DataFrame([info])], ignore_index=True)

                    # 打印
                    """
                    输出的坐标是一个包含四个值的元组 (x0,y0,x1,y1)，
                    分别代表图像矩形框的左下角 (x0, y0) 和右上角 (x1, y1) 的坐标
                    """
        #             print(f"Image {image_filename}; coordinates: {image_rect}; center_coord: {center_coord}")
                    i = i+1
                except Image.DecompressionBombError as e:
                    print(f"Large image deteced, skipping: {e}")
                    
            else:
                print(f"Failed to extract image from xref {img_xref}")
            
    if i < len(pdf_file) + 5:
        print("Anomaly found in PDF {} ({} pages), only {} pics.".format(pdf_file_name, len(pdf_file),i))
    
    # Close the PDF
    pdf_file.close()
    
    # 保存坐标表格
    df_filepath = os.path.join(subfolder_name, "coordinate_df.csv")
    coordinate_df.to_csv(df_filepath)
    
    return coordinate_df

In [7]:
def get_file_path(directory):
    files = [f for f in os.listdir(directory) if f != '.DS_Store']
    return [os.path.join(directory, file) for file in files if os.path.isfile(os.path.join(directory, file))]

In [8]:
# 读取PDF
file_bundle = get_file_path("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2022")
# file_bundle = get_file_path("/Users/improvise/Downloads/test")

# 保存到：
subfolder_name = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2022/02 PyMu_img"
# subfolder_name = "/Users/improvise/Downloads/test_output"

In [9]:
# df_img = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2021/03 PyMu_img_cord/img_cord_1.xlsx")
# file_bundle_mask = df_img['file_name'].apply(lambda x: x.split('_')[0]).unique().tolist()
# file_bundle_name = [os.path.basename(x) for x in file_bundle]
# file_bundle_new = [x for x in file_bundle_name if x not in file_bundle_mask]

# directory = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021"
# file_bundle = [os.path.join(directory, x) for x in file_bundle_new]

In [11]:
img_cord_filelist = get_file_path("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2021/03 PyMu_img_cord")
directory = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021"

for img_cord_path in img_cord_filelist:
    df_img_temp = pd.read_excel(img_cord_path)
    file_bundle_mask = df_img_temp['file_name'].apply(lambda x: x.split('_')[0]).unique().tolist()
    file_bundle_name = [os.path.basename(x) for x in file_bundle]
    file_bundle_new = [x for x in file_bundle_name if x not in file_bundle_mask]
    file_bundle = [os.path.join(directory, x) for x in file_bundle_new]
    
file_bundle

['/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021/002236.SZ-大华股份-大华股份 2021年大华股份环境、社会及管治报告-2022-04-23.pdf',
 '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021/600023.SH-浙能电力-2021年度社会责任暨环境、社会和公司治理(ESG)报告-2022-04-29.pdf',
 '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021/000961.SZ-中南建设-中南建设 关于2021年环境、社会及管治报告的更正公告-2022-04-27.pdf']

In [10]:
def main(times, batch_size = 30, file_bundle = file_bundle,
        img_info_bundle_path = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2022/03 PyMu_img_cord",
        directory = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2022"):
    
    # 定义coordinate_df
    coordinate_df = pd.DataFrame(columns=["file_name","x0","y0","x1","y1","centre_coordinate"])
    
    # 移除已经运行过的PDF
    img_cord_filelist = get_file_path(img_info_bundle_path)
    if img_cord_filelist != []:
        for img_cord_path in img_cord_filelist:
            df_img_temp = pd.read_excel(img_cord_path)
            file_bundle_mask = df_img_temp['file_name'].apply(lambda x: x.split('_')[0]).unique().tolist()
#             print("Remove list: {}".format(file_bundle_mask))
            
            file_bundle_name = [os.path.basename(x) for x in file_bundle]
            file_bundle_new = [x for x in file_bundle_name if x not in file_bundle_mask]
            file_bundle = [os.path.join(directory, x) for x in file_bundle_new]
    
    print("{} PDF files remaining.".format(len(file_bundle)))
    # 取出一个batch_size
    if len(file_bundle) > batch_size:
        file_bundle_batch = file_bundle[0:batch_size]
    else:
        print("\nLast {} files.\n".format(len(file_bundle)))
        file_bundle_batch = file_bundle
    
    print(file_bundle_batch)
    
    # 取图片
    for file in tqdm(file_bundle_batch):
        file_name = os.path.basename(file)
#         print("\nProcessing file:{}".format(file_name))
        try:
            coordinate_df = pd.concat([coordinate_df, PyMuPDF_get_img(file, subfolder_name)],
                                         ignore_index=True)
        except OSError as e: 
            print(f"Error processing file {file}: {e}")
            coordinate_df = pd.concat([coordinate_df, pd.DataFrame([{}.fromkeys(['file_name', 'x0', 'y0', 'x1', 'y1', 'centre_coordinate'],
                                                                 [file_name, None])])],
                                                                 ignore_index=True)
    
    #保存
    coordinate_df.to_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2022/03 PyMu_img_cord/img_cord{}.xlsx".format(times),
                          index = False)
    return coordinate_df

In [11]:
main(14, batch_size = 49)

13 PDF files remaining.

Last 13 files.

['/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2022/600496.SH-精工钢构-精工钢构 精工钢构2022年度社会责任报告暨ESG报告-2023-04-18.pdf', '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2022/601611.SH-中国核建-中国核建 中国核建2022年环境、社会和公司治理(ESG)报告-2023-04-29.pdf', '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2022/601226.SH-华电重工-华电重工 2022年度社会责任暨ESG报告-2023-04-21.pdf', '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2022/301193.SZ-家联科技-家联科技 董事会环境、社会与治理(ESG)委员会议事规则-2023-03-23.pdf', '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2022/600866.SH-星湖科技-星湖科技 广东肇庆星湖生物科技股份有限公司2022年环境、社会及公司治理报告-2023-03-28.pdf', '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2022/002930.SZ-宏川智慧-宏川智慧 2022年度环境、社会及公司治理报告-2023-04-28.pdf', '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2022/002706.SZ-良

  0%|                                                    | 0/13 [00:00<?, ?it/s]

Anomaly found in PDF 600496.SH-精工钢构-精工钢构 精工钢构2022年度社会责任报告暨ESG报告-2023-04-18.pdf (14 pages), only 0 pics.


 15%|██████▊                                     | 2/13 [00:00<00:01,  6.48it/s]

Anomaly found in PDF 601611.SH-中国核建-中国核建 中国核建2022年环境、社会和公司治理(ESG)报告-2023-04-29.pdf (44 pages), only 0 pics.
Anomaly found in PDF 601226.SH-华电重工-华电重工 2022年度社会责任暨ESG报告-2023-04-21.pdf (22 pages), only 0 pics.
Anomaly found in PDF 301193.SZ-家联科技-家联科技 董事会环境、社会与治理(ESG)委员会议事规则-2023-03-23.pdf (5 pages), only 0 pics.


 46%|████████████████████▎                       | 6/13 [03:08<05:09, 44.22s/it]

Anomaly found in PDF 002930.SZ-宏川智慧-宏川智慧 2022年度环境、社会及公司治理报告-2023-04-28.pdf (38 pages), only 38 pics.


100%|███████████████████████████████████████████| 13/13 [13:12<00:00, 60.97s/it]


,file_name,x0,y0,x1,y1,centre_coordinate
0,600866.SH-星湖科技-星湖科技 广东肇庆星湖生物科技股份有限公司2022年环境、社会...,0.000000,-0.000061,595.200012,807.839966,"(297.6000061035156, 403.9199523925781)"
1,600866.SH-星湖科技-星湖科技 广东肇庆星湖生物科技股份有限公司2022年环境、社会...,-0.118851,-0.102661,1190.760864,807.977112,"(595.3210067749023, 403.9372253417969)"
2,600866.SH-星湖科技-星湖科技 广东肇庆星湖生物科技股份有限公司2022年环境、社会...,-0.992258,-0.663818,1190.693604,195.809982,"(594.8506727218628, 97.57308197021484)"
3,600866.SH-星湖科技-星湖科技 广东肇庆星湖生物科技股份有限公司2022年环境、社会...,-162.000000,606.240051,667.200012,849.600037,"(252.60000610351562, 727.9200439453125)"
4,600866.SH-星湖科技-星湖科技 广东肇庆星湖生物科技股份有限公司2022年环境、社会...,71.040001,706.800049,1400.640015,880.800049,"(735.8400077819824, 793.800048828125)"
...,...,...,...,...,...,...
783,301327.SZ-华宝新能-华宝新能 2022年度环境、社会及治理(ESG)报告-2023...,717.165344,186.141708,1102.677124,360.482544,"(909.9212341308594, 273.31212615966797)"
784,301327.SZ-华宝新能-华宝新能 2022年度环境、社会及治理(ESG)报告-2023...,116.929123,484.403778,510.236206,737.343933,"(313.5826644897461, 610.8738555908203)"
785,301327.SZ-华宝新能-华宝新能 2022年度环境、社会及治理(ESG)报告-2023...,116.929146,191.812012,509.182709,486.788269,"(313.0559272766113, 339.3001403808594)"
786,301327.SZ-华宝新能-华宝新能 2022年度环境、社会及治理(ESG)报告-2023...,303.448822,68.492340,510.126587,223.463379,"(406.78770446777344, 145.9778594970703)"


In [23]:
for file in tqdm(file_bundle):
    print("\nProcessing file:{}".format(file))
    try:
        coordinate_df = pd.concat([coordinate_df, PyMuPDF_get_img(file, subfolder_name)],
                                     ignore_index=True)
    except OSError as e: 
        print(f"Error processing file {file}: {e}")
        
coordinate_df.to_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2021/03 PyMu_img_cord/img_cord{}.xlsx".format(times),
                      index = False)

  0%|                                                   | 0/138 [00:00<?, ?it/s]


Processing file:/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021/002236.SZ-大华股份-大华股份 2021年大华股份环境、社会及管治报告-2022-04-23.pdf


  1%|▎                                          | 1/138 [00:03<07:12,  3.16s/it]

Error processing file /Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021/002236.SZ-大华股份-大华股份 2021年大华股份环境、社会及管治报告-2022-04-23.pdf: cannot identify image file <_io.BytesIO object at 0x283d62ea0>

Processing file:/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021/603986.SH-兆易创新-兆易创新2021年环境、社会及管治报告-2022-08-27.pdf


  1%|▌                                        | 2/138 [01:35<2:06:25, 55.77s/it]

...

Anomaly found in PDF 600315.SH-上海家化-上海家化2021年环境、社会及公司治理(ESG)报告-2022-03-17.pdf (44 pages), only 43 pics.

Processing file:/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021/601881.SH-中国银河-中国银河 2021年度社会责任暨环境、社会及管治报告-2022-03-31.pdf


 99%|██████████████████████████████████████▋| 137/138 [2:09:15<00:24, 24.09s/it]


Processing file:/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/01_ESGReport_PDF/2021/688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ESG)报告-2022-05-31.pdf


100%|███████████████████████████████████████| 138/138 [2:12:07<00:00, 57.44s/it]


,file_name,x0,y0,x1,y1,centre_coordinate
0,603986.SH-兆易创新-兆易创新2021年环境、社会及管治报告-2022-08-27....,50.553928,690.151550,152.250702,791.848328,"(101.40231513977051, 740.9999389648438)"
1,603986.SH-兆易创新-兆易创新2021年环境、社会及管治报告-2022-08-27....,154.743622,-8.583344,596.383179,285.843018,"(375.5634002685547, 138.6298370361328)"
2,603986.SH-兆易创新-兆易创新2021年环境、社会及管治报告-2022-08-27....,727.712341,7.994659,1169.351807,302.421021,"(948.5320739746094, 155.2078399658203)"
3,603986.SH-兆易创新-兆易创新2021年环境、社会及管治报告-2022-08-27....,485.679138,73.391815,1191.550049,555.814697,"(838.6145935058594, 314.60325622558594)"
4,603986.SH-兆易创新-兆易创新2021年环境、社会及管治报告-2022-08-27....,-212.346176,51.832214,1931.482178,1020.465820,"(859.568000793457, 536.1490173339844)"
...,...,...,...,...,...,...
11833,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,-161.705566,350.324829,758.925537,868.179810,"(298.6099853515625, 609.2523193359375)"
11834,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,955.273438,630.266846,1153.862427,779.208618,"(1054.5679321289062, 704.7377319335938)"
11835,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,-307.635742,-18.469360,1490.263428,939.090881,"(591.3138427734375, 460.3107604980469)"
11836,688248.SH-南网科技-南方电网电力科技股份有限公司2021年度环境、社会及管治(ES...,-1849.897461,10.169312,295.418701,852.444519,"(-777.2393798828125, 431.3069152832031)"


In [12]:
# try:
#     for file in tqdm(file_bundle):
#         print("\nProcessing file:{}".format(file))
# #         try:
#         coordinate_df = pd.concat([coordinate_df, PyMuPDF_get_img(file, subfolder_name)],
#                                      ignore_index=True)
# #         except Exception as e:
# #             print('====Unexpected Anomaly====\n{}'.format(e))
# except OSError as e:
#     print(f"Error processing file {file}: {e}")

# coordinate_df

In [13]:
# coordinate_df.to_csv("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/PyMu_img01//coordinate_df2.csv")
# coordinate_df.to_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/ESG 2021/03 PyMu_img_cord/img_cord_2.xlsx",
#                       index = False)